In [20]:
import scipy
import numpy as np
from math import atan2, degrees, radians, sqrt
from pyproj import Geod

# Load the .mat file with squeeze_me and struct_as_record options
mat_data = scipy.io.loadmat('AIRS_40KM_2022/20220101_AIRS_3DST-1_40km_grid.mat')
mat_data_info = scipy.io.whosmat('AIRS_40KM_2022/20220101_AIRS_3DST-1_40km_grid.mat')


In [7]:
airs_nh = mat_data['Airs']['NH']
airs_sh = mat_data['Airs']['SH']
airs_alt = mat_data['Airs']['Alt']
airs_x = mat_data['Airs']['x'][0][0][0]
airs_y = mat_data['Airs']['y'][0][0][0]
airs_date = mat_data['Airs']['Date']

In [38]:
mat_data['Airs']['NH']

SyntaxError: unterminated string literal (detected at line 1) (217610903.py, line 1)

In [30]:
f = h5py.File('AIRS_40KM_2022/20220101_AIRS_3DST-1_40km_grid.mat','r+')


OSError: Unable to open file (file signature not found)

In [14]:
# airs_nh_night_tp = mat_data['Airs']['NH']['Night']['tp']
airs_nh = mat_data['Airs']['NH']


In [19]:
np.shape(airs_nh[0][0][0])

(1,)

In [204]:
import math

def km_to_deg(km, radius = 6371.0):
    """
    Convert a distance in kilometers to degrees based on the radius of a sphere.

    Args:
        km (float): Distance in kilometers.
        radius (float): Radius of the sphere in kilometers.

    Returns:
        float: Equivalent distance in degrees.
    """
    earth_radius_km = 6371.0  # mean radius of the Earth in kilometers
    angle = km / earth_radius_km  # angle subtended by the distance at the center of the Earth
    deg = math.degrees(angle * radius / earth_radius_km)
    return deg


km2deg = np.vectorize(km_to_deg)


In [205]:
# Original MATLAB Code
'''
[Airs.X,Airs.Y] = ndgrid(Airs.x,Airs.y);
[Airs.NH.Lat,Airs.NH.Lon] = reckon( 89.999,0,km2deg(sqrt(Airs.X.^2 + Airs.Y.^2)),atan2d(Airs.X,Airs.Y)); 
[Airs.SH.Lat,Airs.SH.Lon] = reckon(-89.999,0,km2deg(sqrt(Airs.X.^2 + Airs.Y.^2)),atan2d(Airs.X,Airs.Y)); 
'''

g = Geod(ellps='clrk66')

Airs = {}
# create ndgrid using meshgrid
Airs_X, Airs_Y = np.meshgrid(airs_x, airs_y)

# calculate latitude and longitude of NH and SH
distance = km2deg((np.sqrt(np.square(Airs_X) + np.square(Airs_Y))))
azimuth = np.degrees(np.arctan2(Airs_X, Airs_Y))

# nh_lat, nh_lon ,_ = g.fwd(90, 0, distance, azimuth)
# sh_lat, sh_lon ,_ = g.fwd(-90, 0, distance, azimuth)

# # assign the calculated values to the dictionary
# Airs['NH'] = {'Lat': nh_lat, 'Lon': nh_lon}
# Airs['SH'] = {'Lat': sh_lat, 'Lon': sh_lon}

In [221]:


def reckon(distance, azimuth):
   nh_lat, nh_lon, _ = g.fwd(90, 0, distance, azimuth)  # ignore third value
   return nh_lat, nh_lon

reckon_v = np.vectorize(reckon)

result = reckon_v(distance,azimuth)
np_result = np.asarray(result)

In [231]:
np_result.reverse().shape()

AttributeError: 'numpy.ndarray' object has no attribute 'reverse'

In [235]:
lat = np_result[0]
lon = np_result[1]

In [239]:
lon

array([[ 0.00073792,  0.00073422,  0.00073051, ..., -0.00073051,
        -0.00073422, -0.00073792],
       [ 0.00073297,  0.00072925,  0.00072552, ..., -0.00072552,
        -0.00072925, -0.00073297],
       [ 0.00072803,  0.00072428,  0.00072053, ..., -0.00072053,
        -0.00072428, -0.00072803],
       ...,
       [ 0.00024433,  0.00024225,  0.00024018, ..., -0.00024018,
        -0.00024225, -0.00024433],
       [ 0.00024516,  0.00024308,  0.00024101, ..., -0.00024101,
        -0.00024308, -0.00024516],
       [ 0.00024597,  0.00024391,  0.00024185, ..., -0.00024185,
        -0.00024391, -0.00024597]])